In [ ]:
import tensorflow as tf
import numpy as np
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense, Activation , LSTM , Dropout , AveragePooling3D
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
df=pd.read_csv('data.csv')
df.head()

In [ ]:
X = df.values[:, 1:-1].astype(np.float32)
y = df.values[:, -1].astype(int) - 1      

Y = to_categorical(y, num_classes=5)


X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.20, random_state=1
)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_lstm = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
X_test_lstm = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))


print("X_train_scaled shape:", X_train_scaled.shape)
print("X_test_scaled shape:", X_test_scaled.shape)

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(X[1,:],label='1')
plt.plot(X[7,:],label='2')
plt.plot(X[12,:],label='3')
plt.plot(X[0,:],label='4')
plt.plot(X[2,:],label='5')
plt.legend()
plt.show()

In [ ]:
model = Sequential()
model.add(LSTM(56, input_shape=(178,1), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(56))
model.add(Dropout(0.3))
model.add(Dense(20))
model.add(Activation('tanh'))
model.add(Dense(5))
model.add(Activation('softmax'))

model.summary()

In [ ]:
from keras.callbacks import EarlyStopping


early_stop = EarlyStopping(
    monitor='val_accuracy',    
    patience=10,           
    restore_best_weights=True,  
    verbose=1
)


model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)


hist = model.fit(
    X_train_scaled,
    Y_train,
    epochs=50,
    batch_size=10,
    validation_data=(X_test_scaled, Y_test),
    callbacks=[early_stop],    
    verbose=2
)